# Main Notebook

In [1]:
import os
import gc
import pytz
import operator
import numpy as np
import pickle as pkl
import xgboost as xgb
from time import sleep
from datetime import datetime
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.python.keras import callbacks
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.losses import mean_absolute_error
from tensorflow.python.keras.layers import Dense, Input, Activation
from tensorflow.python.keras.layers import BatchNormalization, Add, Dropout
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU
from tensorflow.python.keras.optimizers import Adam, Adadelta, SGD

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action = 'ignore', category = FutureWarning)
warnings.filterwarnings(action = 'ignore', category = DeprecationWarning)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import sys
sys.path.append('..')
from utils.generate_features import *

In [2]:
datetime.now(pytz.timezone('Europe/Oslo')).strftime('%m.%d.%Y_%H.%M.%S')

'08.13.2019_15.03.14'

## General Methods

In [3]:
def memory_optimization(dfs):
    for df in dfs:
        del df
    gc.collect()

## Split Training and Validation

In [4]:
def split(df_train):
    train_X, validation_X = train_test_split(df_train, test_size = 0.1, random_state = 0)

    train_X = train_X.reset_index()
    validation_X = validation_X.reset_index()

    train_y = train_X['scalar_coupling_constant']
    train_y = train_y.replace([np.inf, -np.inf], np.nan)
    train_y = train_y.reset_index()
    train_y = train_y.drop(['index'], axis = 1)
    validation_y = validation_X['scalar_coupling_constant']
    validation_y = validation_y.replace([np.inf, -np.inf], np.nan)
    validation_y = validation_y.reset_index()
    validation_y = validation_y.drop(['index'], axis = 1)

    train_X = train_X.drop('scalar_coupling_constant', axis = 1)
    validation_X = validation_X.drop('scalar_coupling_constant', axis = 1)
    
    train_X = train_X.drop(['index'], axis = 1)
    validation_X = validation_X.drop(['index'], axis = 1)
    
    return train_X, train_y, validation_X, validation_y

## NN Training

In [5]:
def create_nn_model(input_shape):
    # input layer
    inp = Input(shape = (input_shape,))

    # first hidden layer
    x = Dense(256, kernel_initializer = 'he_normal')(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # second hidden layer
    x = Dense(512, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # third hidden layer
    x = Dense(1024, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # fourth hidden layer
    x = Dense(512, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # fifth hidden layer
    x = Dense(256, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # sixth hidden layer
    x = Dense(128, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # seventh hidden layer
    x = Dense(128, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # eight hidden layer
    x = Dense(64, kernel_initializer = 'he_normal')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha = 0.05)(x)
    x = Dropout(0.2)(x)
    
    # output layer scalar_coupling_constant
    out = Dense(1, activation = 'linear')(x)   
    model = Model(inputs = inp, outputs = out)
    return model

In [6]:
def nn_train(coupling_type, train_X, train_y, validation_X, validation_y):
    epoch_n = 2000
    verbose = 1
    batch_size = 2048
    model_name = f'../models/nn/coupling_model_{coupling_type}_NN.hdf5'
    
    nn_model = create_nn_model(train_X.shape[1])

    nn_model.compile(loss = 'mae', optimizer = Adam())
    
    es = callbacks.EarlyStopping(monitor = 'loss', min_delta = 0.00001, patience = 64,
                                 verbose = verbose, mode = 'auto', restore_best_weights = True)
    
    rlr = callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience = 32,
                                      min_lr = 1e-7, mode = 'auto', verbose = verbose)
    
    sv_mod = callbacks.ModelCheckpoint(model_name, monitor = 'val_loss', save_best_only = True,
                                       save_weights_only = True, restore_best_weights = True)
    
    history = nn_model.fit(x = train_X.values, y = train_y['scalar_coupling_constant'].values, 
                           validation_data = (validation_X.values, validation_y['scalar_coupling_constant'].values),
                           callbacks = [es, rlr, sv_mod], epochs = epoch_n, batch_size = batch_size, verbose = verbose)
    
    nn_model.save_weights(model_name)

    return nn_model

## XGB Training

In [7]:
def xgb_train(coupling_type, train_X, train_y, validation_X, validation_y):
    model_name_wrt = f'../models/xgb/coupling_model_{coupling_type}_XGB.hdf5'

    xgb_model = xgb.XGBRegressor(base_score = 0.5, booster = 'gbtree', colsample_bylevel = 1,
                                 colsample_bytree = 1, gamma = 0, importance_type = 'gain',
                                 learning_rate = 0.1, max_delta_step = 0, max_depth = 9,
                                 min_child_weight = 1, missing = None, n_estimators = 10000, n_jobs = -1,
                                 nthread = None, objective = 'reg:squarederror', random_state = 101, reg_alpha = 2,
                                 reg_lambda = 0.2, scale_pos_weight = 1, seed = None, silent = False, subsample = 1)

    xgb_model.fit(train_X, train_y, eval_set = [(validation_X, validation_y)], eval_metric = 'mae', 
                  early_stopping_rounds = 32, verbose = True)   
    
    xgb_model.save_model(model_name_wrt)
    #joblib.dump(xgb_model, model_name_wrt)
    
    return xgb_model

In [8]:
def importance(xgb_model, train_X):
    input_features = train_X.columns.values
    feat_imp = xgb_model.feature_importances_
    np.split(feat_imp, len(input_features))
    
    feat_imp_dict = {}
    for i in range(0, len(input_features)):
        feat_imp_dict[feat_imp[i]] = input_features[i]

    sorted_feats = sorted(feat_imp_dict.items(), key = operator.itemgetter(0))
    for i in range(len(sorted_feats) - 1, 0, -1):
        print(sorted_feats[i])

## Training, Blending and Submission

In [9]:
def val_blending_list(nn_val_predict, xgb_val_predict, validation_y):
    nn_val_pred_error  = (validation_y - nn_val_predict)
    xgb_val_pred_error = (validation_y - xgb_val_predict)
    
    val_error_corr = np.corrcoef(nn_val_pred_error, xgb_val_pred_error)
    print(f'Error correlation: {val_error_corr[0][1]}. Error difference {xgb_accuracy - nn_accuracy}')
    
    log_accuracy = 0
    log_accuracy_list = []
    val_predict = np.array([])

    for alpha in np.arange(0, 1.1, 0.1):
        nn_val_predict_scaled = alpha * nn_val_predict
        xgb_val_predict_scaled = (1 - alpha) * xgb_val_predict
        val_predict = nn_val_predict_scaled + xgb_val_predict_scaled
        log_accuracy = np.log(np.mean(np.abs(validation_y - val_predict)))
        log_accuracy_list.append(log_accuracy)

    return log_accuracy_list

In [10]:
load = False

df_test = pd.read_csv('../submissions/submission_best.csv')
test_prediction = df_test['scalar_coupling_constant']
df_test_full = pd.read_csv('../input/test.csv')

start_time = datetime.now()

val_score = {
    '1JHC': np.inf, '1JHN': np.inf, '2JHH': np.inf, '2JHC': np.inf, 
    '2JHN': np.inf, '3JHH': np.inf, '3JHC': np.inf, '3JHN': np.inf
}

coupling_types = ['1JHN', '1JHC', '2JHH', '2JHC', '2JHN', '3JHH', '3JHC', '3JHN']

'''
for coupling_type in coupling_types:
    time = datetime.now(pytz.timezone('Europe/Oslo')).strftime('%m.%d.%Y_%H.%M.%S')
    print(f'Predicting {coupling_type} out of {coupling_types} at {time}.')
    
    df_train, df_test = get_features(coupling_type)
    train_X, train_y, validation_X, validation_y = split(df_train)
    
    ################################# NN #################################
    
    if load:         
        model_name_rd_nn = f'models/nn/nn_model_{coupling_type}.hdf5'
        nn_model = create_nn_model(validation_X.shape[1]) # vals
        print(f'Loading weights from {model_name_rd_nn}.')
        nn_model.load_weights(model_name_rd_nn)
    else:
        nn_model = nn_train(coupling_type, train_X, train_y, validation_X, validation_y)
    
    nn_val_predict = nn_model.predict(validation_X)

    nn_accuracy = np.log(np.mean(np.abs(validation_y.values - nn_val_predict)))
    print(f'Validation score for {coupling_type} is {nn_accuracy} with NN.\n')
    
    ################################# XGB #################################

    if load:
        model_name_rd_xgb = f'..models/xgb/featurebook_{coupling_type}.joblib.dat'

        xgb_model = xgb.XGBRegressor(base_score = 0.5, booster = 'gbtree', colsample_bylevel = 1,
                                     colsample_bytree = 1, gamma = 0, importance_type = 'gain',
                                     learning_rate = 0.1, max_delta_step = 0, max_depth = 9,
                                     min_child_weight = 1, missing = None, n_estimators = 10000, n_jobs = -1,
                                     nthread = None, objective = 'reg:squarederror', random_state = 101, reg_alpha = 2,
                                     reg_lambda = 0.2, scale_pos_weight = 1, seed = None, silent = False, subsample = 1)

        print(f'Loading weights from {model_name_rd_xgb}.')
        #xgb_model.load_model(model_name_rd_xgb)
        xgb_model= joblib.load(model_name_rd_xgb)
    else:
        xgb_model = xgb_train(coupling_type, train_X, train_y, validation_X, validation_y)   
        
    xgb_val_predict = xgb_model.predict(validation_X)
    
    memory_optimization([train_X, train_y, validation_X])
    gc.collect()
    
    validation_y = validation_y['scalar_coupling_constant'].values

    diff = validation_y - xgb_val_predict
    xgb_accuracy = np.log(np.mean(np.abs(diff)))

    print(f'Validation score for {coupling_type} is {xgb_accuracy} with XGB.\n')
     
    ################################# BLEND #################################
        
    nn_val_predict = np.transpose(nn_val_predict[:,0])
    log_accuracy_list = val_blending_list(nn_val_predict, xgb_val_predict, validation_y)
    print(f'Accuracy_list with alphas for {coupling_type}:\n {log_accuracy_list}')
    
    alpha_i = np.argmin(log_accuracy_list)
    log_accuracy = log_accuracy_list[alpha_i]
    val_score[coupling_type] = (log_accuracy)
    alpha = alpha_i * 0.1
    print(f'Blending with alpha = {alpha}, final accuracy for {coupling_type} = {log_accuracy}.') 
    
    ################################# PREDICT #################################
    
    print('Predicting NN:')
    nn_test_predict = nn_model.predict(df_test)
    nn_test_predict_scaled = alpha * nn_test_predict
    nn_test_predict_scaled = np.transpose(nn_test_predict_scaled[:, 0])
    
    print('Predicting XGB:')
    xgb_test_predict = xgb_model.predict(df_test)
    xgb_test_predict_scaled = (1 - alpha) * xgb_test_predict
    
    test_predict = nn_test_predict_scaled + xgb_test_predict_scaled
    test_prediction[df_test_full['type'] == coupling_type] = test_predict

    memory_optimization([df_test, nn_model, nn_val_predict, xgb_model, xgb_val_predict, nn_test_predict, 
                         nn_test_predict_scaled, xgb_test_predict, xgb_test_predict_scaled, test_predict])
    gc.collect()
    
val_score_total = sum(val_score.values()) / len(val_score.keys())
print(f'Total cv score is {val_score_total}.')
'''

for coupling_type in coupling_types:
    df_train, df_test = get_features(coupling_type)
    train_X, train_y, validation_X, validation_y = split(df_train)
    
    xgb_val_predict = xgb_model.predict(validation_X)

NameError: name 'xgb_model' is not defined

## Make submission csv

In [ ]:
def submit(predictions):
    submit = pd.read_csv('submissions/submission_best.csv')  
    submit['scalar_coupling_constant'] = predictions
    submit.to_csv('submissions/submission_blended_full.csv', index = False)

In [ ]:
submit(test_prediction)

time = datetime.now(pytz.timezone('Europe/Oslo')).strftime('%m.%d.%Y_%H.%M.%S')
print(f'Notebook EoF reached at {time} and submission saved.')